In [3]:
# ------------------------------------------------------
# a. Import Required Libraries
# ------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

# ------------------------------------------------------
# b. Upload / Access the Dataset
# ------------------------------------------------------
data = pd.read_csv("C:/Users/UMESH/OneDrive/Desktop/creditcard.csv")

# Separate features and labels
X = data.drop("Class", axis=1).values
y = data["Class"].values

# Normalize
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ------------------------------------------------------
# c. Encoder: Convert Input to Latent Space
# ------------------------------------------------------
input_dim = X_scaled.shape[1]
latent_dim = 4

input_layer = Input(shape=(input_dim,))
encoder_layer1 = Dense(8, activation='relu')(input_layer)
encoder_output = Dense(latent_dim, activation='relu')(encoder_layer1)

# ------------------------------------------------------
# d. Decoder: Reconstruct the Input
# ------------------------------------------------------
decoder_layer1 = Dense(8, activation='relu')(encoder_output)
decoder_output = Dense(input_dim, activation='sigmoid')(decoder_layer1)

# Full Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoder_output)

# ------------------------------------------------------
# e. Compile the Model (Optimizer, Loss, Metrics)
# ------------------------------------------------------
autoencoder.compile(
    optimizer=SGD(learning_rate=0.01),
    loss='mse',
    metrics=['mae']
)

autoencoder.summary()

# ------------------------------------------------------
# Train Only on Normal Data (Class 0)
# ------------------------------------------------------
X_normal = X_scaled[y == 0]

history = autoencoder.fit(
    X_normal, X_normal,
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# ------------------------------------------------------
# Anomaly Detection
# ------------------------------------------------------

# Reconstruct entire dataset
reconstructed = autoencoder.predict(X_scaled)

# Reconstruction Error
mse = np.mean(np.square(X_scaled - reconstructed), axis=1)

# Threshold (Top 95 percentile)
threshold = np.percentile(mse, 95)
print("Threshold:", threshold)

# Predict anomalies
y_pred = (mse > threshold).astype(int)

# ------------------------------------------------------
# Evaluation
# ------------------------------------------------------
print("\nClassification Report:")
print(classification_report(y, y_pred))


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense_8 (Dense)             (None, 8)                 248       
                                                                 
 dense_9 (Dense)             (None, 4)                 36        
                                                                 
 dense_10 (Dense)            (None, 8)                 40        
                                                                 
 dense_11 (Dense)            (None, 30)                270       
                                                                 
Total params: 594 (2.32 KB)
Trainable params: 594 (2.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
7997/7997 [========